# LangChain Basics Demo

This notebook provides a beginner-friendly introduction to using [LangChain](https://python.langchain.com/) for interacting with Large Language Models (LLMs).

We will show how to set up the environment, create a simple prompt, and invoke an LLM using the helper utilities from this repository.

## Installation

Make sure you have the dependencies installed. If you're running this notebook for the first time, uncomment and run the cell below.

In [ ]:
# !pip install -r requirements.txt

## Setting up API keys

Create a `.env` file in the project root or set the environment variables directly in the notebook. At minimum you need to specify the `llm_provider` and its corresponding API key. Below is an example using environment variables.

In [ ]:
import os
os.environ['llm_provider'] = 'mistral'  # 'mistral', 'gemini', or 'openai'
os.environ['mistral_api_key'] = 'YOUR_MISTRAL_API_KEY'
# os.environ['gemini_api_key'] = 'YOUR_GEMINI_API_KEY'
# os.environ['openai_api_key'] = 'YOUR_OPENAI_API_KEY'

## Using `LLMFactory`

The `LLMFactory` class from this repository chooses the correct LangChain wrapper based on the provider. We'll use it to generate a short summary about any topic.

In [ ]:
from util.llm_factory import LLMFactory
from util.system_prompt import prompt_generate_summary

text = 'Explain the importance of clean energy in a few sentences.'
response = LLMFactory.invoke(
    system_prompt=prompt_generate_summary,
    human_message=text,
    temperature=0.7,
)
print(response.content.strip())

## Basic LangChain Components

Behind the scenes, `LLMFactory.invoke` creates a prompt using `ChatPromptTemplate` and sends it to the selected model. Below is a minimal example without the factory.

In [ ]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI

# Use your preferred model wrapper
chat = ChatOpenAI(api_key=os.environ['mistral_api_key'], model='gpt-3.5-turbo', temperature=0.7)

system_tmpl = SystemMessagePromptTemplate.from_template('You are a helpful assistant.')
human_tmpl = HumanMessagePromptTemplate.from_template('What is LangChain?')

prompt = ChatPromptTemplate.from_messages([system_tmpl, human_tmpl])
messages = prompt.format_messages()
response = chat.invoke(messages)
print(response.content.strip())

This demonstrates the basic building blocks: prompt templates, LLM wrappers, and calling `invoke`. You can extend this with chains, agents, or memory for more advanced workflows.

## Building a Simple Chain

Let's create a tiny `LLMChain` that takes a topic and returns a single sentence description.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Template with a variable placeholder
pt = PromptTemplate.from_template('Describe {topic} in one sentence.')
chain = LLMChain(prompt=pt, llm=LLMFactory.create_llm_instance())

res = chain.invoke({'topic': 'quantum computing'})
print(res['text'].strip())

## Conversational Memory

LangChain makes it easy to store chat history. Below we use `ConversationBufferMemory` to remember what we talked about.

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

memory = ConversationBufferMemory()
chat_chain = ConversationChain(llm=LLMFactory.create_llm_instance(), memory=memory)

print(chat_chain.invoke('Hello there!')['response'])
print(chat_chain.invoke('What did I just say?')['response'])

## Conclusion

This notebook introduced basic LangChain usage: installing dependencies, configuring API keys, invoking an LLM, building prompts, creating chains, and managing conversation history.